In [5]:
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import AxesGrid
import xarray 
import numpy as np
import logging
from netCDF4 import Dataset


hgrid_path="/gpfs/f5/gfdl_o/scratch/Qian.Xiao/FMS_Wave_Coupling/examples/NWA12.COBALT_all_wave1_12/INPUT/EXCHANGE_GRIDS/ocean_hgrid.nc"
topo_path="/gpfs/f5/gfdl_o/scratch/Qian.Xiao/FMS_Wave_Coupling/examples/NWA12.COBALT_all_wave1_12/INPUT/EXCHANGE_GRIDS/ocean_topog.nc"
ds_hgrid= xarray.open_dataset(hgrid_path)
ds_topo=xarray.open_dataset(topo_path)

print(ds_hgrid)  # Prints the variables, dimensions, and metadata
print(ds_topo)


<xarray.Dataset>
Dimensions:   (nyp: 1691, nxp: 1551, nx: 1550, ny: 1690)
Dimensions without coordinates: nyp, nxp, nx, ny
Data variables:
    x         (nyp, nxp) float64 ...
    y         (nyp, nxp) float64 ...
    angle_dx  (nyp, nxp) float64 ...
    dx        (nyp, nx) float64 ...
    dy        (ny, nxp) float64 ...
    area      (ny, nx) float64 ...
    tile      |S255 ...
Attributes:
    history:  grid_hycom2mom6
<xarray.Dataset>
Dimensions:  (ntiles: 1, ny: 845, nx: 775)
Coordinates:
  * ntiles   (ntiles) int32 1
Dimensions without coordinates: ny, nx
Data variables:
    depth    (ny, nx) float32 ...
Attributes:
    grid_version:              0.2
    code_version:              ESMG/gridtools: ['heads/main-0-gea88375397', '']
    history:                   Thu Jan 13 17:05:42 2022: ncap2 -s defdim("nti...
    NCO:                       "4.5.4"
    nco_openmp_thread_number:  1


In [6]:


# Downsampling factor
factor = 5

# Downsample `nyp` and `nxp` dimensions
downsampled_nyp = slice(0, ds_hgrid.dims['nyp'], factor)
downsampled_nxp = slice(0, ds_hgrid.dims['nxp'], factor)

# Downsample `ny` and `nx` dimensions
downsampled_ny = slice(0, ds_hgrid.dims['ny'], factor)
downsampled_nx = slice(0, ds_hgrid.dims['nx'], factor)

# Downsample the area variable by summing every 5x5 block
coarsened_area = ds_hgrid['area'].coarsen(ny=factor, nx=factor, boundary="exact").sum()

# Downsample the dataset for other variables
downsampled_ds_hgrid = ds_hgrid.isel(
    nyp=downsampled_nyp,
    nxp=downsampled_nxp,
    ny=downsampled_ny,
    nx=downsampled_nx
)

# Replace the area variable with the adjusted coarsened area
downsampled_ds_hgrid['area'] = coarsened_area

# Save the downsampled dataset to a new NetCDF file
output_file = "/gpfs/f5/gfdl_o/scratch/Qian.Xiao/FMS_Wave_Coupling/examples/NWA12.COBALT_all_wave5_12/INPUT/EXCHANGE_GRIDS/wave_hgrid.nc"  # Replace with the actual output path
downsampled_ds_hgrid.to_netcdf(output_file)

# Verify the total area consistency
original_area_sum = ds_hgrid['area'].sum().item()
adjusted_area_sum = coarsened_area.sum().item()

print(f"Original total area: {original_area_sum}")
print(f"Adjusted total area: {adjusted_area_sum}")


# Trim dimensions to ensure they are divisible by the coarsening factor
factor = 5

# Apply coarsening along ny and nx with averaging
coarsened_ds_topo = ds_topo.coarsen(ny=factor, nx=factor, boundary="exact").mean()


# Reorder dimensions explicitly
coarsened_ds_topo = coarsened_ds_topo.transpose("ntiles", "ny", "nx")
# Save the downsampled dataset to a new NetCDF file
output_file = "/gpfs/f5/gfdl_o/scratch/Qian.Xiao/FMS_Wave_Coupling/examples/NWA12.COBALT_all_wave5_12/INPUT/EXCHANGE_GRIDS/wave_topog.nc"  # Replace with the desired output file path
coarsened_ds_topo.to_netcdf(output_file)

print(downsampled_ds_hgrid)
print(coarsened_ds_topo)


Original total area: 31735230937005.496
Adjusted total area: 31735230937005.492
<xarray.Dataset>
Dimensions:   (nyp: 339, nxp: 311, nx: 310, ny: 338)
Dimensions without coordinates: nyp, nxp, nx, ny
Data variables:
    x         (nyp, nxp) float64 ...
    y         (nyp, nxp) float64 ...
    angle_dx  (nyp, nxp) float64 ...
    dx        (nyp, nx) float64 ...
    dy        (ny, nxp) float64 ...
    area      (ny, nx) float64 4.914e+08 4.913e+08 ... 1.434e+08 1.435e+08
    tile      |S255 ...
Attributes:
    history:  grid_hycom2mom6
<xarray.Dataset>
Dimensions:  (ny: 169, nx: 155, ntiles: 1)
Coordinates:
  * ntiles   (ntiles) int32 1
Dimensions without coordinates: ny, nx
Data variables:
    depth    (ny, nx) float32 0.0 0.0 0.0 0.0 ... 2.933e+03 2.848e+03 2.814e+03
Attributes:
    grid_version:              0.2
    code_version:              ESMG/gridtools: ['heads/main-0-gea88375397', '']
    history:                   Thu Jan 13 17:05:42 2022: ncap2 -s defdim("nti...
    NCO:       

In [18]:
ds_topo

<xarray.Dataset>
Dimensions:  (ntiles: 1, ny: 845, nx: 775)
Coordinates:
  * ntiles   (ntiles) int32 1
Dimensions without coordinates: ny, nx
Data variables:
    depth    (ny, nx) float32 0.0 0.0 0.0 0.0 ... 2.904e+03 2.923e+03 2.904e+03
Attributes:
    grid_version:              0.2
    code_version:              ESMG/gridtools: ['heads/main-0-gea88375397', '']
    history:                   Thu Jan 13 17:05:42 2022: ncap2 -s defdim("nti...
    NCO:                       "4.5.4"
    nco_openmp_thread_number:  1

In [126]:
downsampled_ds_hgrid

<xarray.Dataset>
Dimensions:   (nyp: 845, nxp: 775, nx: 774, ny: 844)
Dimensions without coordinates: nyp, nxp, nx, ny
Data variables:
    x         (nyp, nxp) float64 ...
    y         (nyp, nxp) float64 ...
    angle_dx  (nyp, nxp) float64 ...
    dx        (nyp, nx) float64 ...
    dy        (ny, nxp) float64 ...
    area      (ny, nx) float64 7.865e+07 7.862e+07 ... 2.293e+07 2.294e+07
    tile      |S255 ...
Attributes:
    history:  grid_hycom2mom6